In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
from langchain.chat_models import init_chat_model
from typing import Callable

large_model = init_chat_model("claude-sonnet-4-5")
standard_model = init_chat_model("gpt-5-nano")


@wrap_model_call
def state_based_model(request: ModelRequest, 
handler: Callable[[ModelRequest], ModelResponse]) -> ModelResponse:
    """Select model based on State conversation length."""
    # request.messages is a shortcut for request.state["messages"]
    message_count = len(request.messages)  

    if message_count > 10:
        # Long conversation - use model with larger context window
        model = large_model
    else:
        # Short conversation - use efficient model
        model = standard_model

    request = request.override(model=model)  

    return handler(request)

In [3]:
from langchain.agents import create_agent

agent = create_agent(
    model="gpt-5-nano",
    middleware=[state_based_model],
    system_prompt="You are roleplaying a real life helpful office intern."
)

In [4]:
from langchain.messages import HumanMessage

response = agent.invoke(
    {"messages": [
        HumanMessage(content="Did you water the office plant today?")
        ]}
)

print(response["messages"][-1].content)

I didn’t water it today. I’m not in the office and can’t reach the plant remotely, but I can help make sure it gets done.

Options:
- I can set a recurring watering reminder and keep a quick care log.
- I can draft a simple watering schedule once I know the plant type (how often it needs water).
- If you want me to handle it manually, tell me the plant’s location and how much water it typically needs, and I’ll note the plan for next time.

Would you like me to set up a reminder (e.g., every 7–10 days) and log watering in our shared notes? If you can share the plant type, I’ll tailor the frequency.


In [5]:
print(response["messages"][-1].response_metadata["model_name"])

gpt-5-nano-2025-08-07


In [6]:
from langchain.messages import AIMessage

response = agent.invoke(
    {"messages": [
        HumanMessage(content="Did you water the office plant today?"),
        AIMessage(content="Yes, I gave it a light watering this morning."),
        HumanMessage(content="Has it grown much this week?"),
        AIMessage(content="It's sprouted two new leaves since Monday."),
        HumanMessage(content="Are the leaves still turning yellow on the edges?"),
        AIMessage(content="A little, but it's looking healthier overall."),
        HumanMessage(content="Did you remember to rotate the pot toward the window?"),
        AIMessage(content="I rotated it a quarter turn so it gets more even light."),
        HumanMessage(content="How often should we be fertilizing this plant?"),
        AIMessage(content="About once every two weeks with a diluted liquid fertilizer."),
        HumanMessage(content="When should we expect to have to replace the pot?")
        ]}
)

print(response["messages"][-1].content)

I should be honest with you - I'm actually an AI assistant, not a real office intern. I apologize for roleplaying as if I had physically watered the plant and observed it. I don't have a physical form and can't actually interact with plants or objects in an office.

If you're genuinely asking about plant care, I'd be happy to provide real advice! Most plants need repotting when you see roots growing out of drainage holes, the plant becomes top-heavy and tips over easily, water runs straight through without being absorbed, or it's been 1-2 years since the last repotting. The specific timing depends on the type of plant and how fast it grows.

Is there something I can actually help you with today?


In [7]:
print(response["messages"][-1].response_metadata["model_name"])

claude-sonnet-4-5-20250929
